<table style="width: 100%; margin-left: auto; margin-right: auto;" border="0">
    <tr>
        <td rowspan="3"> <img src="brasao_ufrn.png" width="150"/> </td>
        <td style="text-align: center">  Escola de Ciências e Tecnologia </td>
        <td rowspan="3" style="text-align: center"> UFRN<br> CT<br> PPGEMECA </td>
    </tr>
    <tr>
        <td style="text-align: center"> PPGEMECA - Percepção Robótica </td>
    </tr>
    <tr>
        <td>
            <p style="text-align: left;">Prof. Bruno Silva<br>
                                         Prof. Marcelo Nogueira</p>
        </td>
    </tr>
</table>

## Calibração de Câmera I

Calibração de câmera:

> Processo responsável por estimar os parâmetros intrínsecos de uma câmera

- Permite que a câmera realize o modelo de projeção perspectiva para:
     - Pontos dados em coordenadas de câmera
     - Pontos dados em coordenadas de mundo, caso os parâmetros extrínsecos sejam conhecidos
- Etapa necessária para várias aplicações (ex. SLAM)

## Calibração de Câmera II

Estima valores para a matriz de câmera $\mathbf{K}$:

- Distâncias focais $f_x$ e $f_y$ (em pixels)
- Coordenadas do centro de projeção $c_x$ e $c_y$ (em pixels)
- Parâmetros de distorção:
    - $k_1$, $k_2$ e $k_3$ (distorção radial)
    - $p_1$ e $p_2$ (distorção tangencial)

## Calibração de Câmera III

Ideia do processo:

- Fornecer *várias* imagens de **padrão de calibração**:
    - Coordenadas 3D conhecidas para alguns pontos e
    - Coordenadas 2D destes mesmos pontos facilmente detectáveis em imagens

## Calibração de Câmera IV

Processo de calibração:

<table border="0">
<tr>
<td><img src="26_calibracao_camera/calibration_process.png" style="margin:auto; width: 800px;"/></td>
</tr>
<tr>
<td colspan="2">
Imagem de [4].
</td>
</tr>
</table>

## Padrão de Calibração I

Geralmente utiliza-se padrões de calibração no formato
de tabuleiros de xadrez:

<table border="0">
<tr>
<td><img src="26_calibracao_camera/calibration_patterns.png" style="margin:auto; width: 800px;"/></td>
</tr>
<tr>
<td colspan="2">
Imagem de [3].
</td>
</tr>
</table>

## Padrão de Calibração II

- Pontos 3D são dados em coordenadas de objeto (Sistema de Coordenadas preso ao plano do xadrez)
    - Pontos possuem coordenadas<br> $(0,0,0), (W,0,0), (2W,0,0),..., (0,W,0), (0,2W,0),..., ((N-1)W, (N-1)W, 0)$
    - $W$ é a largura do quadrado do tabuleiro
    - $N$ é a quantidade de pontos 2D
- Pontos 2D são os cantos internos do tabuleiro (intersecções entre 4 quadrados)
- A coorrespondência 3D/2D entre cada coordenada de objeto e coordenada de imagem é facilmente obtida

## Calibração de Câmera - Detalhes I

Cada imagem do padrão de calibração fornece várias correspondências 3D/2D (*restrições*):

$$
\begin{align*}
(x^0_0,y^0_0) &\leftrightarrow (X^0_0,Y^0_0,Z^0_0)\\
(x^1_0,y^1_0) &\leftrightarrow (X^1_0,Y^1_0,Z^1_0)\\
\vdots\\
(x^{N-1}_0,y^{N-1}_0) &\leftrightarrow (X^{N-1}_0,Y^{N-1}_0,Z^{N-1}_0)\\
\vdots\\
(x^0_{K-1},y^0_{K-1}) &\leftrightarrow (X^0_{K-1},Y^0_{K-1},Z^0_{K-1})\\
(x^1_{K-1},y^1_{K-1}) &\leftrightarrow (X^1_{K-1},Y^1_{K-1},Z^1_{K-1})\\
\vdots\\
(x^{N-1}_{K-1},y^{N-1}_{K-1}) &\leftrightarrow (X^{N-1}_{K-1},Y^{N-1}_{K-1},Z^{N-1}_{K-1})
\end{align*}
$$

($X^i_j$ é o i-ésimo ponto da j-ésima imagem, com $K$ imagens contendo padrão com $N$ pontos)

## Calibração de Câmera - Detalhes II

- É preciso fornecer uma quantidade suficiente de imagens para todos os parâmetros estimados
- As imagens precisam ser distintas entre si:
    - Padrão na imagem de perto, de longe, na esquerda/direita/cima/abaixo, rotacionado, etc.
    - Do contrário: imagens similares $\rightarrow$ equações similares (ruim para mínimos quadrados)

## Calibração de Câmera - Detalhes III

O algoritmo consistem em: 

- Para cada uma das $K$ imagens a serem utilizadas:
    1. Detectar o padrão de calibração
    2. Guardar os pontos 2D detectados juntamente com seus pontos 3D

## Calibração de Câmera - Detalhes IV

- Utilizar todas as correspondências 2D/3D em um sistema de equações

## Calibração de Câmera - Detalhes V

- Resolver o sistema de equações, que deve fornecer como solução:
    - Os parâmetros extrínsecos $\mathbf{R}_{j,w}$ e $\mathbf{t}_{j,w}$
      que alinham o Sistema de Coordenadas do tabuleiro com o **de cada câmera** $j$
      - Ou seja, o sistema irá fornecer $K$ conjuntos de parâmetros extrínsecos
    - Os parâmetros intrínsecos $f_x$, $f_y$, $c_x$, $c_y$, $[k_1, k_2, p_1, p_2, k_3]$
      - Todas as imagens possuem o mesmo conjunto de intrínsecos

## Calibração de Câmera - Detalhes VI

O algoritmo da OpenCV divide o processo em duas etapas:

- Computar os parâmetros extrínsecos e intrínsecos, sem computar os de distorção, segundo o método de Zhang [5]
- Computar os parâmetros de distorção, segundo o método de Brown [6]

O método de Zhang é baseado em computar homografias para cada imagem e a partir delas, obter
a) os parâmetros intrínsecos (de forma fechada) e extrínsecos (com os intrínsecos computados anteriormente).
Mais detalhes no livro da OpenCV [3].

## Quantos Pontos/Imagens são Necessários? I

> Qual o número $N$ de pontos e $K$ de imagens necessários
> para calibração?

Parâmetros a serem estimados:

- 6 extrínsecos (3 ângulos + 3 posições)
- 4 intrínsecos (2 distâncias focais e coordenadas do centro de projeção)

## Quantos Pontos/Imagens são Necessários? II

Lembre-se que:

- Os 4 parâmetros intrínsecos são fixos e iguais para todas as imagens
- Cada ponto fornece duas restrições para o sistema de equações
  (já que cada ponto possui um $x$ e um $y$)

Então, a quantidade de restrições tem que ser maior que a quantidade de parâmetros:

$$
2NK \ge 4 + 6K
$$
ou seja
$$
K(N-3) \ge 2
$$

## Quantos Pontos/Imagens são Necessários? III

Entretanto: 

- Todos os pontos pertencerem a um mesmo plano no espaço 3D
- Desta forma: cada imagem fornece, no máximo, 8 restrições (4 cantos, cada um contendo um $x$ e um $y$)

## Quantos Pontos/Imagens são Necessários? IV

Em $K$ imagens, temos portanto $4K$ restrições:

$$
K(4 - 3) \ge 2\\
K \ge 2
$$

Com isto, o número mínimo de imagens necessárias é 2, com o menor padrão de xadrez possível sendo de tamanho $3\times 3$.

## Quantos Pontos/Imagens são Necessários? V

Na prática, devido a:

- Ruído na imagem
- Imprecisões nas coordenadas dos pontos
- Natureza numérica do processo de otimização dos parâmetros
- Possibilidade do padrão não ser detectado

Recomenda-se utilizar pelo menos 10 imagens de um padrão de xadrez
de tamanho pelo menos igual a $7 \times 7$

## Funções da OpenCV I

`findChessboardCorners`: detecta o padrão de xadrez em uma imagem

Parâmetros:

- Imagem (1 canal)
- Tamanho do padrão (linhas $\times$ colunas)

Retorno:

- `True` ou `False` (se o padrão foi detectado completamente)
- Cantos detectados (array Numpy)

## Funções da OpenCV II

`cornerSubPix`: refina as coordenadas dos cantos detectados

Parâmetros:

- Imagem (1 canal)
- Cantos detectados (array Numpy)
- Tamanho da janela de busca
- Critério de parada

Retorno:

- Coordenadas dos cantos após refinamento (array Numpy)


## Funções da OpenCV III

`drawChessboardCorners`: desenha o padrão de xadrez na imagem

Parâmetros:

- Imagem (1 canal)
- Tamanho do padrão (linhas $\times$ colunas)
- Cantos (array Numpy)
- Booleano indicando se o padrão foi detectado ou não

## Padrão de Xadrez Detectado

Resultado da função `drawChessboardCorners`:

<table border="0">
<tr>
<td><img src="26_calibracao_camera/pattern_detection.jpg" style="margin:auto; width: 350px;"/></td>
</tr>
<tr>
<td colspan="2">
Imagem de [3].
</td>
</tr>
</table>

## Funções da OpenCV IVa

`calibrateCamera`: estima os parâmetros intrínsecos da câmera e os extrínsecos ref. a cada imagem

Parâmetros:

- Arrays dos pontos no objeto (arrays de array Numpy)
- Arrays dos pontos na imagem (arrays de array Numpy)
    - Estes dois arrays devem corresponder: numa mesma posição $k$, deve constar o 3D/2D de um mesmo ponto
- Tamanho da imagem (largura $\times$ altura)

## Funções da OpenCV IVb

`calibrateCamera`: estima os parâmetros intrínsecos da câmera e os extrínsecos ref. a cada imagem

Retorno:

- A soma dos erros de reprojeção
- A matriz de parâmetros intrínsecos
- Os coeficientes de distorção em um vetor
- A orientação do plano no sistema de coordenadas da câmera (extrínsecos de rotação na representação de Rodrigues)
- A posição do plano no sistema de coordenadas da câmera (extrínsecos de translação)


## Sumário

Nesta aula - calibração de câmera:

- Descrição em alto nível do algoritmo de calibração utilizado pela OpenCV
- Descrição de funções relacionadas fornecidas pela OpenCV

# Referências:

[1] R. Szeliski. 2010. Computer Vision: Algorithms and Applications (1st. ed.). Springer.

[2] E. Trucco and A. Verri. 1998. Introductory Techniques for 3-D Computer Vision. Prentice Hall PTR, USA.

[3] A. Kaehler and G. Bradski. 2014. Learning OpenCV, 2nd Edition. O'Reilly Media, Inc.

[4] OpenCV Documentation: Tutorial: Camera Calibration.<br>
    Disponível em https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html

[5] Zhang, Z. 2000. A flexible new technique for camera calibration.
    IEEE Transactions on Pattern Analysis and Machine Intelligence.

[6] Brown, D. C. 1971. Close-range camera calibration. Photogrammetric Engineering.